<a href="https://colab.research.google.com/github/silviosnjr/Fase3_TechChallenge/blob/main/Fase3_TechChallenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3ª fase do TechChallenge | Pos Tech | IA para Dev's
## Silvio Sales do Nascimento Junior (RM 353303)

Vídeo no Youtube sobre o TechChallenge [https://youtu.be/8c6EZHQRCeI](https://youtu.be/8c6EZHQRCeI)

Conectando ao Google drive onde está o dataset

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Instalando bibliotecas que podem ser necessárias

In [3]:
%pip install beautifulsoup4
%pip install requests
%pip install torch
%pip install pandas
%pip install scikit-learn
%pip install transformers

## 1. Escolha do Dataset
Não foi possível fazer tratamento de uma base de dados tão extensa como a do `trn.json`. Por isso fiz uma extração da base de dados com 15 mil registro o arquivo está no [drive](https://drive.google.com/file/d/1a47o65lY_XsGDHY2ORaB75j0cB2-2WIX/view?usp=drive_link).

In [4]:
file_path = '/content/drive/My Drive/Fase3_TechChallenge/dataset/trn_15k.json'

## 2. Preparação do Dataset
### 2.1 Carregar e visualizar os dados
Carregando o arquivo trn15k.json para visualizá-lo e entender sua estrutura:

In [5]:
import pandas as pd

# Carregar o arquivo JSON
df = pd.read_json(file_path, lines=True)

# Ver as primeiras linhas do dataset
print(df.head())

                                               title  \
0         Yellow Box Women's Yulisa Flip Flop Sandal   
1                   Nine West Women's Eastnor Sandal   
2  Roma Costume Women's 1 Piece Santa's Envy-Red ...   
3  1 Box of 100 Sony 1.55v Silver Oxide Watch Bat...   
4  Winning Angels: The 7 Fundamentals of Early St...   

                                             content  
0                                                     
1                                                     
2                                                     
3  100 Sony 377 Watch Batteries Silver Oxide, Mer...  
4  "Winning Angels is a superbly organised and in...  


### 2.2. Limpeza e pré-processamento dos dados
Limpar e pré-processar os dados conforme necessário para o modelo escolhido.

In [6]:
import re

# Função para limpeza do texto (remoção de caracteres especiais, conversão para minúsculas, etc.)
def limpar_texto(texto):
    texto = texto.lower()  # Converter para minúsculas
    texto = re.sub(r'[^a-zA-Z0-9\s]', '', texto)  # Remover caracteres especiais
    texto = re.sub(r'\s+', ' ', texto).strip()  # Remover espaços extras
    return texto

# Aplicar a limpeza nas colunas 'title' e 'content'
df['title_clean'] = df['title'].apply(limpar_texto)
df['content_clean'] = df['content'].apply(limpar_texto)

# Visualizar os dados limpos
print(df[['title_clean', 'content_clean']].head())


                                         title_clean  \
0          yellow box womens yulisa flip flop sandal   
1                    nine west womens eastnor sandal   
2   roma costume womens 1 piece santas envyred white   
3  1 box of 100 sony 155v silver oxide watch batt...   
4  winning angels the 7 fundamentals of early sta...   

                                       content_clean  
0                                                     
1                                                     
2                                                     
3  100 sony 377 watch batteries silver oxide merc...  
4  winning angels is a superbly organised and inv...  


### 2.3 Preparar os prompts para o fine-tuning

In [7]:
# Criar prompts para o fine-tuning usando o título e o conteúdo
df['prompt'] = 'Título: ' + df['title_clean'] + '\nDescrição:'
df['resposta'] = df['content_clean']

# Exibir alguns prompts preparados
print(df[['prompt', 'resposta']].head())

                                              prompt  \
0  Título: yellow box womens yulisa flip flop san...   
1  Título: nine west womens eastnor sandal\nDescr...   
2  Título: roma costume womens 1 piece santas env...   
3  Título: 1 box of 100 sony 155v silver oxide wa...   
4  Título: winning angels the 7 fundamentals of e...   

                                            resposta  
0                                                     
1                                                     
2                                                     
3  100 sony 377 watch batteries silver oxide merc...  
4  winning angels is a superbly organised and inv...  


## 3. Chamada do Foundation Model
### 3.1 Importando o modelo BERT pré-treinado:

In [8]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# Carregar o tokenizer e o modelo pré-treinado BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Definir dispositivo (GPU ou CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoin

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

### 3.2 Rodar um teste com o modelo pré-treinado:

In [9]:
# Função para fazer o teste com o modelo pré-treinado
def test_model_on_sample(prompt):
    model.eval()  # Colocar o modelo em modo de avaliação
    with torch.no_grad():
        # Tokenizar o texto de exemplo
        encoding = tokenizer(
            prompt,
            truncation=True,
            padding='max_length',
            max_length=128,
            return_tensors='pt'
        ).to(device)

        # Fazer uma previsão com o modelo pré-treinado
        outputs = model(**encoding)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        return predictions.item()

# Testar o modelo com um exemplo do dataset
prompt_exemplo = "Title: Disney Pixar? \nDescription: Disney Pixar"
resultado = test_model_on_sample(prompt_exemplo)

print("Resultado da previsão antes do fine-tuning:", resultado)

Resultado da previsão antes do fine-tuning: 0


# 4. Execução do Fine-Tuning


In [10]:
!pip install datasets

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.3 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.


In [11]:
from datasets import Dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments

# Função para tokenizar os prompts e adicionar rótulos
def tokenize_function(examples):
    return tokenizer(examples['prompt'], truncation=True, padding='max_length', max_length=128)

# Certifique-se de que você tem uma coluna de rótulos no seu dataframe, por exemplo, 'label'
# Se os rótulos não existirem, você precisará adicioná-los de acordo com a sua tarefa de classificação.
df['label'] = df['resposta'].apply(lambda x: 1 if "relevante" in x else 0)  # Exemplo de criação de rótulos

# Converter o DataFrame para Dataset Hugging Face
dataset = Dataset.from_pandas(df)
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Definir os parâmetros de treinamento
training_args = TrainingArguments(
    output_dir='./results',          # Diretório para salvar os resultados
    num_train_epochs=3,              # Número de épocas
    per_device_train_batch_size=16,  # Tamanho do batch de treino
    per_device_eval_batch_size=64,   # Tamanho do batch de avaliação
    warmup_steps=500,                # Passos de aquecimento para o otimizador
    weight_decay=0.01,               # Taxa de decaimento de peso para regularização
    logging_dir='./logs',            # Diretório para salvar logs
    logging_steps=10,                # Log a cada 10 passos
    evaluation_strategy="epoch"      # Avaliação no final de cada época
)

# Carregar o modelo pré-treinado BERT
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Configurar o Trainer com rótulos
trainer = Trainer(
    model=model,                         # O modelo BERT a ser ajustado
    args=training_args,                  # Parâmetros de treinamento definidos acima
    train_dataset=dataset,               # Dataset de treino
    eval_dataset=dataset,                # Dataset de avaliação (pode separar dados de validação se necessário)
)

# Executar o fine-tuning
trainer.train()

# Salvar o modelo ajustado
model.save_pretrained("/content/drive/My Drive/Fase3_TechChallenge/fine_tuned_bert")


Map:   0%|          | 0/15000 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,0.000000,0.000011
2,0.000000,0.000004
3,0.000000,0.000003


### 5. Geração de Respostas
Configure o modelo treinado para receber perguntas dos usuários. O modelo deverá gerar uma resposta baseada na pergunta do usuário e nos dados provenientes do _fine-tuning_, incluindo as fontes fornecidas.

In [10]:
import pandas as pd
from transformers import BertTokenizer
import torch

# Carregar o dataset preparado
file_path = '/content/drive/My Drive/Fase3_TechChallenge/dataset/trn_15k.json'
df = pd.read_json(file_path, lines=True)

# Carregar o modelo e o tokenizer
model = BertForSequenceClassification.from_pretrained("/content/drive/My Drive/Fase3_TechChallenge/fine_tuned_bert")
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Colocar o modelo em modo de avaliação
model.eval()

# Função para buscar a informação mais relevante no dataset
def buscar_resposta(pergunta):
    # Tokenizar a pergunta do usuário
    inputs = tokenizer(pergunta, return_tensors="pt", truncation=True, padding=True, max_length=128)

    # Fazer a previsão com o modelo fine-tuned
    with torch.no_grad():
        outputs = model(**inputs)

    # Extrair a saída da previsão (logits)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=-1).item()

    # Procurar correspondências no dataset com base na pergunta
    resultados = df[df['title'].str.contains(pergunta, case=False, na=False)]

    if not resultados.empty:
        # Retornar a descrição correspondente
        return resultados.iloc[0]['content']  # Retorna o primeiro resultado encontrado
    else:
        return "Desculpe, não consegui encontrar uma resposta relevante no dataset."

# Exemplo de pergunta em inglês
pergunta_usuario = "Lord of the Rings?"
resposta = buscar_resposta(pergunta_usuario)

# Exibir a resposta
print(f"Pergunta: {pergunta_usuario}")
print(f"Resposta: {resposta}")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Pergunta: Lord of the Rings?
Resposta: Ian Brodie is one of New Zealand's most successful non-fiction writers of recent years. The Lord of the Rings Location Guidebook was one of the inaugural Booksellers Platinum Bestsellers, with international sales of over 450,000 copies. Ian lives in Matamata where he is the Media and Communications Manager for Hobbiton Movie Set. He is also the author of the highly successful aviation series, Warbirds over Wanaka, and in 2005 he was awarded an MNZM for services to tourism. A devoted Tolkien fanatic, keen photographer and movie fan, he is currently working on another movie-related book.


In [12]:
pergunta_usuario = "Brazil?"
resposta = buscar_resposta(pergunta_usuario)

# Exibir a resposta
print(f"Pergunta: {pergunta_usuario}")
print(f"Resposta: {resposta}")

Pergunta: Brazil?
Resposta: This official 2014 FIFA World Cup wall chart has a complete list of team groups &amp;&nbsp;fixtures so that you can plot the tournament all the way to the final. This&nbsp;wall chart&nbsp;measures 61c


In [16]:
pergunta_usuario = "iPhone?"
resposta = buscar_resposta(pergunta_usuario)

# Exibir a resposta
print(f"Pergunta: {pergunta_usuario}")
print(f"Resposta: {resposta}")

Pergunta: iPhone?
Resposta: Shipped and sold by AccessoryOne!


In [18]:
pergunta_usuario = "Samsung?"
resposta = buscar_resposta(pergunta_usuario)

# Exibir a resposta
print(f"Pergunta: {pergunta_usuario}")
print(f"Resposta: {resposta}")

Pergunta: Samsung?
Resposta: Ships by MoMo Store Packing from US. Usually takes 3-5 business days for delivery in US!!


In [20]:
pergunta_usuario = "FIAP"
resposta = buscar_resposta(pergunta_usuario)

# Exibir a resposta
print(f"Pergunta: {pergunta_usuario}")
print(f"Resposta: {resposta}")

Pergunta: FIAP
Resposta: Desculpe, não consegui encontrar uma resposta relevante no dataset.
